#### Automated Procurement System

In [3]:
# Warning Control 
from warnings import filterwarnings
from pydantic import PydanticDeprecatedSince20

filterwarnings('ignore')  
filterwarnings('ignore', category=PydanticDeprecatedSince20)  

In [4]:
from crewai import Agent,Task, Crew, LLM, Process
from crewai_tools import SerperDevTool
from crewai.tools import tool
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
import os
from dotenv import load_dotenv
import agentops
from pydantic import BaseModel, Field
from scrapegraph_py import Client
import json
from IPython.display import HTML, display

In [5]:
# --------- setup Environment  ----------

load_dotenv()

basic_llm = LLM(model='gpt-4.1-mini', temperature=0 )

agentops.init(
    api_key = os.getenv("AGENTOPS_API_KEY"),
    skip_auto_end_session= True
)
scrape_client = Client(api_key = os.getenv("SGAI_API_KEY"))

🖇 AgentOps: You're on the agentops free plan 🤔


In [6]:
output_dir = "./ai-agent-output"
os.makedirs(output_dir,exist_ok=True)


company_context = StringKnowledgeSource(
    content= "Rankyx is a company that provides AI solutions to help websites refine "
             "their search and recommendation systems."
)

#### CrewAI Tools

In [7]:
search_tool = SerperDevTool()   # To search the web 

#### Creating Pydantic Objects

In [8]:
# ---- Pydantic Models (structured outputs) for Task1 ----

class SuggestedSearchQueries(BaseModel):
    queries: list[str] = Field(..., 
                              title = "Suggested Queries",
                              description = "Suggested search queries to be passed to the search engine",
                              min_length=1,
                              max_length=10
                              )

In [9]:
# ---- Nested Pydantic Models (structured outputs) for Task2 ----

class SignleSearchResult(BaseModel):
    title: str
    url: str
    content: str
    score: float
    search_query: str

class AllSearchResults(BaseModel):
    results: list[SignleSearchResult]

In [13]:
# ---- Nested Pydantic Models for Task3 ----

class ProductSpec(BaseModel):
    specification_name: str
    specification_value: str

class SingleExtractedProduct(BaseModel):
    page_url: str = Field(..., title="The original url of the product page")
    product_title: str = Field(..., title="The title of the product")
    product_image_url: str = Field(..., title="The url of the product image")
    product_url: str = Field(..., title="The url of the product")
    product_current_price: float = Field(..., title="The current price of the product", ge=0)
    product_original_price: float = Field(..., title="The original price of the product before discount.", ge=0)
    product_discount_percentage: float = Field(title="The discount percentage of the product. Set to None if no discount", default=None)
    
    product_specs: list[ProductSpec] = Field(..., title="The specifications of the product. Focus on the most important specs to compare.", min_length=1, max_length=5)
    
    agent_recommendation_rank: int = Field(..., title="The rank of the product to be considered in the final procurement report. (out of 5, Higher is Better) in the recommendation list ordering from the best to the worst")
    agent_recommendation_notes: list[str] = Field(..., title="A set of notes why would you recommend or not recommend this product to the company, compared to other products.")

class AllExtractedProducts(BaseModel):
    products: list[SingleExtractedProduct]

#### Creating Agents

In [14]:
#--------Agent 1 ----------------

search_queries_recommendation_agent = Agent(
    role = "Search Queries Recommendation Agent",
    goal = "To provide a list of suggested search queries to be passed to the search engine. "
            "The queries must be varied and looking for specific items.",
    backstory= 
            "The agent is designed to help in looking for products by providing a list "
            "of suggested search queries to be passed to the search engine based on "
            "the context provided.",

    llm=basic_llm,
    verbose = True    
)

In [15]:
#-------- Agent 2 ----------------

search_engine_agent = Agent(
    role="Search Engine Agent",
    goal="To search for products based on the suggested search query",
    backstory="The agent is designed to help in looking for products by searching for products based on the suggested search queries.",
    llm=basic_llm,
    verbose=True,
    tools=[search_tool]
)

In [16]:
@tool
def web_scraping_tool(page_url: str):
    """
    An AI Tool to help an agent to scrape a web page.

    Example:
    web_scraping_tool(
        page_url="https://www.amazon.com/Uncanny-Brands-Office-Single-Mifflin/dp/B095QQ5JS4",
    )
    """
    details = scrape_client.smartscraper(
        website_url = page_url,
        user_prompt = "Extract ```json\n" + SingleExtractedProduct.schema_json() + "```\n From the web page"
    )

    return {
        "page_url": page_url,
        "details": details
    }


In [18]:
#-------- Agent 3 ----------------

scraping_agent = Agent(
    role= "Web scraping agent",
    goal= "To extract details from any website",
    backstory= "The agent is designed to help in looking for required values from any website url."
               "These details will be used to decide which best product to buy.",
    llm= basic_llm,
    tools=[web_scraping_tool],
    verbose= True,
)

In [19]:
#-------- Agent 4 ----------------

procurement_report_author_agent = Agent(
    role= "Procurement Report Author Agent",
    goal= "To generate a professional HTML page for the procurement report",
    backstory= "The agent is designed to assist in generating a professional HTML "
               "page for the procurement report after looking into a list of products.",
    llm= basic_llm,
    verbose= True,
)

#### Creating Tasks

In [20]:
#------- Task 1 -----------------

search_queries_recommendation_task = Task(
    description= (
        "Rankyx is looking to buy {product_name} at the best prices (value for a price strategy) "
        "The company target any of these websites to buy from: {websites_list} "
        "The company wants to reach all available products on the internet to be compared "
        "later in another stage. "
        "The stores must sell the product in {country_name} "
        "Generate at maximum {no_keywords} queries."
        "Search keywords could mention specific brands, types or technologies."
        "The search query must reach an ecommerce webpage for product, and not a blog or listing page."
    ),

    expected_output = "A JSON object containing a list of suggested search queries.",
    output_json = SuggestedSearchQueries,
    output_file = os.path.join(output_dir, "step_1_suggested_search_queries.json"),       
    agent = search_queries_recommendation_agent,
)

In [21]:
#------- Task 2 -----------------

search_engine_task = Task(
    description="\n".join([
        "The task is to search for products based on the suggested search queries.",
        "You have to collect results from multiple search queries.",
        "Ignore any suspicious links or not an ecommerce single product website link.",
        "Ignore any search results with confidence score less than ({score_th})",
        "The search results will be used to compare prices of products from different websites."
    ]),
    expected_output= "A JSON object containing the search result.",
    output_json= AllSearchResults,
    output_file= os.path.join(output_dir,"step_2_search_results.json"),
    agent= search_engine_agent
)


In [22]:
#-------- Task 3 ----------------

scraping_task = Task(
    description= (
        "The task is to extract product details from any ecommerce store page url."
        "The task has to collect results from multiple pages urls."
        "Collect the best {top_recommendations_no} products from the search results."
    ),
    expected_output= "A JSON object containing products details",
    output_json= AllExtractedProducts,
    output_file= os.path.join(output_dir, "step_3_extracted_products.json"),
    agent= scraping_agent,
)

In [23]:
#-------- Task 4 ----------------

procurement_report_author_task = Task(
    description="\n".join([
        "The task is to generate a professional HTML page for the procurement report.",
        "You have to use Bootstrap CSS framework for a better UI.",
        "Use the provided context about the company to make a specialized report.",
        "The report will include the search results and prices of products from different websites.",
        "The report should be structured with the following sections:",
        "1. Executive Summary: A brief overview of the procurement process and key findings.",
        "2. Introduction: An introduction to the purpose and scope of the report.",
        "3. Methodology: A description of the methods used to gather and compare prices.",
        "4. Findings: Detailed comparison of prices from different websites, including tables and charts.",
        "5. Analysis: An analysis of the findings, highlighting any significant trends or observations.",
        "6. Recommendations: Suggestions for procurement based on the analysis.",
        "7. Conclusion: A summary of the report and final thoughts.",
        "8. Appendices: Any additional information, such as raw data or supplementary materials."
    ]),
    expected_output="A professional HTML page for the procurement report.",
    output_file=os.path.join(output_dir, "step_4_procurement_report.html"),
    agent=procurement_report_author_agent,
)

#### Creating the Crew

In [24]:
rankyx_crew = Crew(
    agents=[
        search_queries_recommendation_agent,
        search_engine_agent,
        scraping_agent,
        procurement_report_author_agent,
    ],
    tasks=[
        search_queries_recommendation_task,
        search_engine_task,
        scraping_task,
        procurement_report_author_task
    ],
    process=Process.sequential,
    knowledge_sources= [company_context]
)

#### Running the Crew

In [25]:
rankyx_inputs = {
        'product_name': "coffee machine for the office",
        'websites_list': [
                          "www.webstaurantstore.com",
                          "www.amazon.com",
                          "www.uline.com"
                        ],
        'country_name': "USA",
        'no_keywords': 10,
        'score_th':0.10,
        'top_recommendations_no':10
    }

crew_results = rankyx_crew.kickoff( inputs = rankyx_inputs)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Search Queries Recommendation Agent                                                                     │
│                                                                                                                 │
│  Task: Rankyx is looking to buy coffee machine for the office at the best prices (value for a price strategy)   │
│  The company target any of these websites to buy from: ['www.webstaurantstore.com', 'www.amazon.com',           │
│  'www.uline.com'] The company wants to reach all available products on the internet to be compared later in     │
│  another stage. The stores must sell the product in USA Generate at maximum 10 queries.Search keywords could    │
│  mention specific brands, types or technologies.The search query must reach an ecommerce webpage for product,   │
│  and not a blog or listing page.                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Search Queries Recommendation Agent                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "queries": [                                                                                                 │
│      "coffee machine office use site:www.webstaurantstore.com",                                                 │
│      "commercial coffee maker best value site:www.amazon.com",                                                  │
│      "espresso machine office bulk purchase site:www.uline.com",                                                │
│      "Keurig coffee maker office site:www.amazon.com",                                                          │
│      "Drip coffee machine commercial site:www.webstaurantstore.com",                                            │
│      "Nespresso office coffee machine site:www.amazon.com",                                                     │
│      "automatic coffee brewer office site:www.uline.com",                                                       │
│      "coffee machine with grinder office site:www.webstaurantstore.com",                                        │
│      "Hamilton Beach commercial coffee maker site:www.amazon.com",                                              │
│      "office coffee machine under $200 site:www.uline.com"                                                      │
│    ]                                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Search Engine Agent                                                                                     │
│                                                                                                                 │
│  Task: The task is to search for products based on the suggested search queries.                                │
│  You have to collect results from multiple search queries.                                                      │
│  Ignore any suspicious links or not an ecommerce single product website link.                                   │
│  Ignore any search results with confidence score less than (0.1)                                                │
│  The search results will be used to compare prices of products from different websites.                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Search Engine Agent                                                                                     │
│                                                                                                                 │
│  Thought: Thought: I need to perform searches for each of the provided queries on the specified websites to     │
│  gather product information for comparison. I will start with the first query and proceed sequentially.         │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"coffee machine office use site:www.webstaurantstore.com\"}"                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"page_url\": \"https://www.amazon.com/Keurig-K155-Office-Commercial-Single/dp/B00DWKEHU4\"}"                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  I encountered an error while trying to use the tool. This was the error: [402] Insufficient credits.           │
│   Tool web_scraping_tool accepts these inputs: Tool Name: web_scraping_tool                                     │
│  Tool Arguments: {'page_url': {'description': None, 'type': 'str'}}                                             │
│  Tool Description:                                                                                              │
│      An AI Tool to help an agent to scrape a web page.                                                          │
│                                                                                                                 │
│      Example:                                                                                                   │
│      web_scraping_tool(                                                                                         │
│          page_url="https://www.amazon.com/Uncanny-Brands-Office-Single-Mifflin/dp/B095QQ5JS4",                  │
│      )                                                                                                          │
│      .                                                                                                          │
│  Moving on then. I MUST either use a tool (use one at time) OR give my best final answer not both at the same   │
│  time. When responding, I must use the following format:                                                        │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: you should always think about what to do                                                              │
│  Action: the action to take, should be one of [web_scraping_tool]                                               │
│  Action Input: the input to the action, dictionary enclosed in curly braces                                     │
│  Observation: the result of the action                                                                          │
│  ```                                                                                                            │
│  This Thought/Action/Action Input/Result can repeat N times. Once I know the final answer, I must return the    │
│  following format:                                                                                              │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: I now can give a great answer                                                                         │
│  Final Answer: Your final answer must be the great and the most complete as possible, it must be outcome        │
│  described                                                                                                      │
│                                                                                                                 │
│  ```                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web scraping agent                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "products": [                                                                                                │
│      {                                                                                                          │
│        "page_url":                                                                                              │
│  "https://www.webstaurantstore.com/13953/commercial-coffee-makers-brewers.html?srsltid=AfmBOoqfxaoMe570YmxqpDW  │
│  OgKM4w_M2wVDWW-xbYto2V8OIxxxQ7_2s",                                                                            │
│        "product_title": "Blended Reverse Osmosis System with White 10 Gallon Tank 200 GPD",                     │
│        "product_image_url": "https://www.webstaurantstore.com/images/products/medium/179000/2380116.jpg",       │
│        "product_url":                                                                                           │
│  "https://www.webstaurantstore.com/avantco-dhc-26-2-6-gallon-10-liter-hot-beverage-hot-topping-dispenser/177DH  │
│  C26.html",                                                                                                     │
│        "product_current_price": 419.99,                                                                         │
│        "product_original_price": 419.99,                                                                        │
│        "product_discount_percentage": null,                                                                     │
│        "product_specs": [                                                                                       │
│          {"specification_name": "Type", "specification_value": "Commercial Coffee Makers / Brewers"},           │
│          {"specification_name": "Categories", "specification_value": "Restaurant Equipment, Beverage            │
│  Equipment, Smallwares"},                                                                                       │
│          {"specification_name": "Capacity", "specification_value": "2.6 Gallon (10 Liter)"},                    │
│          {"specification_name": "Type", "specification_value": "Hot Beverage / Hot Topping Dispenser"}          │
│        ],                                                                                                       │
│        "agent_recommendation_rank": 3,                                                                          │
│        "agent_recommendation_notes": [                                                                          │
│          "Good for high-volume settings",                                                                       │
│          "Customizable",                                                                                        │
│          "Easy to use"                                                                                          │
│        ]                                                                                                        │
│      },                                                                                                         │
│      {                                                                                                          │
│        "page_url": "https://www.amazon.com/commercial-c

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Procurement Report Author Agent                                                                         │
│                                                                                                                 │
│  Task: The task is to generate a professional HTML page for the procurement report.                             │
│  You have to use Bootstrap CSS framework for a better UI.                                                       │
│  Use the provided context about the company to make a specialized report.                                       │
│  The report will include the search results and prices of products from different websites.                     │
│  The report should be structured with the following sections:                                                   │
│  1. Executive Summary: A brief overview of the procurement process and key findings.                            │
│  2. Introduction: An introduction to the purpose and scope of the report.                                       │
│  3. Methodology: A description of the methods used to gather and compare prices.                                │
│  4. Findings: Detailed comparison of prices from different websites, including tables and charts.               │
│  5. Analysis: An analysis of the findings, highlighting any significant trends or observations.                 │
│  6. Recommendations: Suggestions for procurement based on the analysis.                                         │
│  7. Conclusion: A summary of the report and final thoughts.                                                     │
│  8. Appendices: Any additional information, such as raw data or supplementary materials.                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Procurement Report Author Agent                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```html                                                                                                        │
│  <!DOCTYPE html>                                                                                                │
│  <html lang="en">                                                                                               │
│  <head>                                                                                                         │
│    <meta charset="UTF-8" />                                                                                     │
│    <meta name="viewport" content="width=device-width, initial-scale=1" />                                       │
│    <title>Procurement Report - Office Coffee Machines</title>                                                   │
│    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet" />     │
│    <style>                                                                                                      │
│      body {                                                                                                     │
│        padding-top: 3rem;                                                                                       │
│        padding-bottom: 3rem;                                                                                    │
│        background-color: #f8f9fa;                                                                               │
│      }                                                                                                          │
│      .product-image {                                                                                           │
│        max-width: 100px;                                                                                        │
│        height: auto;                                                                                            │
│      }                                                                                                          │
│      .table thead th {                                                                                          │
│        vertical-align: middle;                                                                                  │
│        text-align: center;                                                                                      │
│      }                                                                                                          │
│      .table tbody td {                                                                                          │
│        vertical-align: middle;                                                                                  │
│        text-align: center;                                                                                      │
│      }                                                                                                          │
│      .chart-container {                                                                                         │
│        width: 100%;                                                                                             │
│        max-width: 700px;                                                                                        │
│        margin: 2rem auto;                              

In [28]:
display(HTML("./ai-agent-output/step_4_procurement_report.html"))

Product Image,Product Title,Supplier,Price (USD),Original Price (USD),Discount (%),Key Specifications,Recommendation Rank
,Blended Reverse Osmosis System with White 10 Gallon Tank 200 GPD,WebstaurantStore,$419.99,$419.99,–,Type: Commercial Coffee Makers / Brewers Capacity: 2.6 Gallon (10 Liter) Category: Restaurant & Beverage Equipment Hot Beverage / Hot Topping Dispenser,3
,Zulay Premium Commercial Coffee Urn,Amazon,$159.99,$169.99,5.88%,Brand: Zulay Material: Stainless Steel Size: 100 Cup Capacity Weight: 20 pounds,5
